In [1]:
import re
import os
import time
from multiprocessing import Process, Queue
import threading
from urllib import request, error, parse
import requests
from tqdm import tqdm
import http.client
import datetime 
import json

In [2]:
from gzip import GzipFile
from io import StringIO, BytesIO
import zlib

In [3]:
class rank_spider:
    def __init__(self):
        self.headers = {"Custom-Source":"GitHub@zszumich","Content-Type": "application/json","Referer": "https://kengxxiao.github.io/Kyouka/"}
        self.url = 'https://service-kjcbcnmw-1254119946.gz.apigw.tencentcs.com/'
        self.payload = {'history':'0'}
    def get_data(self, check_type, refer):
        payload = self.payload
        url = self.url
        if(check_type == 'name'):
            payload['clanName'] = refer
            url = url+'name/0'
        if(check_type == 'rank'):
            url = url+'/rank/{0}'.format(refer)
        if(check_type == 'leader'):
            url = url+'/leader/0'
            payload['leaderName'] = refer
            
        data = requests.post(url, data = json.dumps(payload), headers = self.headers)
        data = json.loads(data.text)
        return(data)
        

In [4]:
my_spider = rank_spider()

In [5]:
html = my_spider.get_data('leader','高坂丽奈')

In [6]:
print(html)

{'code': 0, 'msg': '', 'data': [{'rank': 8486, 'damage': 17237327, 'clan_name': '优花梨的小屋', 'leader_name': '高坂丽奈', 'leader_viewer_id': 1365627307506}], 'ts': 0, 'full': 1}


In [7]:
def check_rank_score(spider, rank):
    data = spider.get_data('rank', rank)
    score = data['data'][0]['damage']
    return(score)

In [8]:
score = check_rank_score(my_spider, 2800)
print(score)

25653461


In [9]:
def check_rank(rank):
    rank_list = [1,3,10,20,50,200,600,1200,2800,5000,10000,15000,25000,40000,60000]
    rank_list = [x for x in rank_list if x <= rank]
    close_rank = rank_list[-1]
    return(close_rank)
        

In [10]:
check_rank(10)

10

In [11]:
def check_status(score):
    score_ratio = [[1, 1, 1.1, 1.1, 1.2],[1.2, 1.2, 1.5, 1.7, 2]]
    boss_hp = [[6000000, 8000000, 10000000, 12000000, 20000000],[6000000, 8000000, 10000000, 12000000, 20000000]]
    num_epoch = 2
    num_boss = len(boss_hp[0])
    damage = 0
    score_c = 0
    count_c = 0
    count_b = 0
    final_damage = 0
    res_damage = 0
    temp = '{0}周目{1}王,进度{2}%'
    while(score_c < score):
        count_b = 0
        while(score_c < score):
            final_damage = damage
            if(count_c < num_epoch-1):
                damage = damage + boss_hp[count_c][count_b]
                score_c = score_c + boss_hp[count_c][count_b]*score_ratio[count_c][count_b]
            else:
                damage = damage + boss_hp[num_epoch-1][count_b]
                score_c = score_c + boss_hp[num_epoch-1][count_b]*score_ratio[num_epoch-1][count_b]
            if(score_c > score):
                break
            else:
                final_damage = damage
                count_b = count_b + 1
            if(count_b == num_boss):
                break
        if(score_c > score):
            break
        count_c = count_c + 1
    remain_score = score_c - score
    if(count_c < num_epoch-1):
        remain_damage = remain_score/score_ratio[count_c][count_b]
        process = int((boss_hp[count_c][count_b]-remain_damage)/boss_hp[count_c][count_b]*100)
    else:
        remain_damage = remain_score/score_ratio[num_epoch-1][count_b]
        process = int((boss_hp[num_epoch-1][count_b]-remain_damage)/boss_hp[num_epoch-1][count_b]*100)
    result = temp.format(count_c+1, count_b+1, process)
    return(result)
        

In [61]:
check_status(811099339)

'10周目2王,进度42%'

In [12]:
def check_hanghui(name):
    result = '数据来自:https://kengxxiao.github.io/Kyouka/\n'
    my_spider = rank_spider()
    data = my_spider.get_data('name',name)
    template ='公会名称: {0}\n会长:{1}\n会长ID:{2}\n会战分数:{3}\n排名:{4}\n进度:{5}\n距档线{6}相差分数:{7}\n'
    if(data['full'] == 0):
        temp = '无法查询到此工会'
        result = result+temp
    else:
        for data_t in data['data']:
            temp = template
            score = data_t['damage']
            rank = data_t['rank']
            dang_xian = check_rank(rank)
            rank_score = check_rank_score(my_spider, dang_xian)
            score_diff = rank_score - score
            process = check_status(score)
            temp = temp.format(data_t['clan_name'],data_t['leader_name'],data_t['leader_viewer_id'],score,rank,process,
                              dang_xian)
            result = result+temp+'\n'
        
    return(result)
            

In [13]:
result = check_hanghui('胖达团')

IndexError: Replacement index 7 out of range for positional args tuple

In [ ]:
print(result)

In [86]:
net_east = ['“生而为人,我很抱歉”','“日推到这首歌的人一定很温柔吧”','“我认识的很多,但我的朋友很少','“陌生人,如果你刷到这条评论就睡吧',
           '“老师讲什么都听不懂了,好想放弃”','“我喜欢你，从满心欢喜到满目疮疾”','“这么晚还在听歌的人,你一定也很寂寞吧”',
           '“温柔吗,拿命换的”','“自己有多伤心,歌单知道,枕头知道,你不知道”','“舔了三年,她终究还是离开了我','“你说爱情要两情相悦,否则,所有的喜欢都是心酸”',
           '“我是两个孩子的父亲…','“《人间失格》里有这样一句话：仅一夜之间，我的心判若两人。他自人山人海中来，原来只为给我一场空欢喜。你来时携风带雨，我无处可避；你走时乱了四季，我久病难医”',
           '“是什么原因让你来听这首歌啊”','“这歌真正听懂的时候一定很悲伤吧”','“我通过了你的好友请求，现在我们可以开始聊天',
           '“你走以后……”','“我今天失恋了，陌生人能给我点个赞吗？”','“教堂的白鸽不会亲吻乌鸦”','“我说下雨了，不是指天气”','“今年12岁，重度抑郁症20年了”',
           '“妈妈在我出生前就走了”','“现实里没有任何一个人理我”','“今天看了看，30楼好像也不高。。。”','“人间不值得”','“人间真好，下次不来了”',
           '“我今年17岁，出意外的话明年18岁，不出意外的话我将永远17岁…”','你也只活了一次，凭什么说我选择的人生是错的。','你爱过一个人吗？从满心欢喜到满目疮痍······',
           '考研可能也失败了，妈妈明天因为心脏有问题要去看，继父不管，我亲爹关机不敢接我电话，那些亲人除了我阿姨在为我们东奔西跑，我才23岁了我承受了那么多，听着这首歌会哭的',
           '重度抑郁啦，不能给家里添麻烦，不买药啦，自己百度怎么缓解情绪。不告诉朋友啦，因为无从开口，平时我都是逗他们开心的。不告诉男朋友啦，"','爸爸在年前查出肺腺癌晚期，一切来的太突然，本来就不富裕的家庭更是雪上加霜，经常看到妈妈夜里擦眼泪，可我到底还是高一学生，分担不了家庭负担，只能好好学习，不知道爸爸还能陪我多久，我真的好爱我的家庭，好爱爸爸和妈妈。',
           '现在已经五年级下册了，老师讲的内容什么也听不进去，成绩也越来越差了，原本想好好努力的，结果没过几天就不想学了，我该怎么办呐',
           '我抑郁症了，我温柔吗，半条命换的快来安慰我我快高考了，赶紧鼓励我','我倒数第一，语文零分，数学零分，英语全是选择题，我成功的错过了所有正确答案。2020我中考，完全没希望······',
           '今年经历太多了，被绿了，被骗2500元，疫情，亲人去世，父母离婚，考试取消，抑郁症复发，控制不住自己的情绪，我也好想爱这个世界啊，我才十几岁我可以的！',
           '五年没有联系的前任，结婚了生了孩子，是我们一直想要的女儿，名字竟然也是我取的,但姓不一样了。','我的爷爷得了冠状病毒，我的生活到了谷底。但是我要坚强，勇敢向前看，迎接我的未来，我的生活。阿门！祝我爷爷康复[可爱][流泪][流泪]',
           '我愛的是位男生，我卻也是一位男生，我的人生，還能成人生嗎?','大四了，工作没着落，没有爱的人，未来不知道去哪，不知道有多少人和我一样，好像渐渐失去直觉，失去了爱人的勇气。好想大声哭一场。我是真的需要个方向啊，更需要人陪。',
           '我逃了很久 拒绝过很多人 直到遇见你 我开始动摇 相信所谓的苦尽甘来 后来才发现你也不爱我','我讨厌圈钱的女孩子，可是如果是你，我更讨厌没钱的自己','他不在对岸 我也不够勇敢',
           '磨脚的鞋子我不会穿第二次，剪坏我头发的理发店，我不会去第二次，可是就是那个不适合我的人我爱了不止一次',
           '你说什么最难熬，是漫漫长夜，还是两月三年，是两人相爱的不能见面，还是你爱的人不爱你','“《人间失格》里有这样一句话：“若能避开猛烈的欢喜，自然也不会有悲痛的来袭。””',
           '“你回信息速度越来越慢，和我说话越来越少，我就知道你要走了”','“我有那么多奇妙的能力 却留不住你”','“这么晚还在听歌的人,你一定也很寂寞吧“',
           '“那些看似没有听懂的回应，大概就是再委婉不过的拒绝”','“《人间失格》里有这么一句话：‘你在此地不要走动，我去给你买袋橘子。’”']

In [87]:
print(len(net_east))

51


In [88]:
import random

In [92]:
t = random.choice(net_east)
print(t)

“现实里没有任何一个人理我”
